<a href="https://colab.research.google.com/github/TamannaAhmad/research-paper-analyser/blob/main/image_caption_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf

In [ ]:
import fitz  # PyMuPDF
import os
import re
from tqdm import tqdm
import numpy as np
from PIL import Image
import io
from google.colab import drive
import shutil
import zipfile
from google.colab import files

In [ ]:
# mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def extract_caption(page, image_rect):
    x0, y0, x1, y1 = image_rect
    page_width = page.rect.width
    page_height = page.rect.height

    # Get all text blocks on the page
    blocks = page.get_text("blocks")

    # Find text blocks that might be captions
    potential_captions = []

    for block in blocks:
        block_rect = fitz.Rect(block[:4])
        block_text = block[4]

        # Skip blocks with very little text
        if len(block_text.strip()) < 5:
            continue

        # Check if block is below or very near the image
        is_below = (block_rect.y0 >= y1 - 5 and
                    block_rect.y0 <= y1 + 100 and
                    max(0, block_rect.x0 - x1) < 50 and
                    max(0, x0 - block_rect.x1) < 50)

        # Check overlap with image column
        x_overlap = (min(block_rect.x1, x1) - max(block_rect.x0, x0))
        has_x_overlap = x_overlap > 0

        # Check if text contains caption indicators
        has_caption_text = (re.search(r'\b(fig|figure)\.?\s*\d+', block_text, re.IGNORECASE) is not None)

        # Either positioned correctly or contains caption text
        if (is_below or has_x_overlap) and has_caption_text:
            # Calculate distance from image
            center_img = ((x0 + x1)/2, (y0 + y1)/2)
            center_block = ((block_rect.x0 + block_rect.x1)/2, (block_rect.y0 + block_rect.y1)/2)
            distance = ((center_img[0] - center_block[0])**2 + (center_img[1] - center_block[1])**2)**0.5

            potential_captions.append((block_text, distance))

    if potential_captions:
        # Sort by distance to image
        potential_captions.sort(key=lambda x: x[1])
        return potential_captions[0][0].strip()

    # Fallback: use broader search with large area below image
    caption_area = fitz.Rect(max(0, x0 - 50), y1, min(page_width, x1 + 50), min(page_height, y1 + 150))
    caption_text = page.get_text("text", clip=caption_area)

    if caption_text:
        # Try to find caption patterns
        match = re.search(r'((?:Figure|Fig\.?)\s*\d+[\\.:]?.*?)(?=\n\n|\Z)', caption_text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(1).strip()

    return None

In [ ]:
def should_merge_images(rect1, rect2, proximity_threshold=50):
    """Determine if two image rectangles should be considered part of the same figure"""
    # Check horizontal alignment
    horizontal_aligned = (abs(rect1.y0 - rect2.y0) < proximity_threshold or
                          abs(rect1.y1 - rect2.y1) < proximity_threshold)

    # Check vertical alignment
    vertical_aligned = (abs(rect1.x0 - rect2.x0) < proximity_threshold or
                        abs(rect1.x1 - rect2.x1) < proximity_threshold)

    # Check proximity
    center1 = ((rect1.x0 + rect1.x1)/2, (rect1.y0 + rect1.y1)/2)
    center2 = ((rect2.x0 + rect2.x1)/2, (rect2.y0 + rect2.y1)/2)
    distance = ((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)**0.5

    # Check overlap
    overlap = not (rect1.x1 < rect2.x0 or rect2.x1 < rect1.x0 or
                  rect1.y1 < rect2.y0 or rect2.y1 < rect1.y0)

    return (horizontal_aligned or vertical_aligned) and (distance < proximity_threshold * 3 or overlap)

In [ ]:
def get_figure_number(caption_text):
    """Extract figure number from caption text"""
    if not caption_text:
        return None

    match = re.search(r'(?:Figure|Fig\.?)\s*(\d+)', caption_text, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return None

In [ ]:
def extract_images(file_path, output_base_dir):
    pdf_file = fitz.open(file_path)
    total_figures = 0  # count total number of figures in PDF
    total_captions = 0  # count total number of captions found

    folder_name = os.path.basename(file_path)[:-4]  # extract PDF name without extension
    folder_path = os.path.join(output_base_dir, folder_name)

    if not os.path.exists(folder_path):  # create folder if it doesn't exist
        os.makedirs(folder_path)

    # Process each page
    for page_index in range(len(pdf_file)):
        page = pdf_file.load_page(page_index)  # load the page
        image_list = page.get_images(full=True)  # get images on the page

        if not image_list:
            continue

        # Get all image rectangles and group them by figure
        image_data = []
        for img in image_list:
            xref = img[0]
            try:
                base_image = pdf_file.extract_image(xref)
                rects = page.get_image_rects(xref)

                if rects:
                    caption_text = extract_caption(page, rects[0])
                    fig_number = get_figure_number(caption_text)

                    image_data.append({
                        'xref': xref,
                        'rect': rects[0],
                        'caption': caption_text,
                        'fig_number': fig_number,
                        'base_image': base_image,
                        'grouped': False
                    })
            except Exception as e:
                print(f"[!] Error analyzing image on page {page_index+1}: {e}")

        # Group images by proximity and caption
        figure_groups = []

        # First, try to group by figure number
        for idx, img_data in enumerate(image_data):
            if img_data['grouped']:
                continue

            group = [idx]
            img_data['grouped'] = True

            # If this image has a figure number, look for other parts with the same number
            if img_data['fig_number'] is not None:
                for j, other_img in enumerate(image_data):
                    if j != idx and not other_img['grouped'] and other_img['fig_number'] == img_data['fig_number']:
                        group.append(j)
                        other_img['grouped'] = True

            figure_groups.append(group)

        # Then try to group remaining images by proximity
        for idx, img_data in enumerate(image_data):
            if img_data['grouped']:
                continue

            group = [idx]
            img_data['grouped'] = True

            for j, other_img in enumerate(image_data):
                if j != idx and not other_img['grouped'] and should_merge_images(img_data['rect'], other_img['rect']):
                    group.append(j)
                    other_img['grouped'] = True

            figure_groups.append(group)

        # Process each figure group
        for group_idx, group in enumerate(figure_groups):
            total_figures += 1

            # Use the caption from the first image in the group
            caption_text = image_data[group[0]]['caption']

            # If we have multiple images in the group
            if len(group) > 1:
                # We'll keep track of the images for combining but not save individual parts
                pil_images = []
                for img_idx in group:
                    img_data = image_data[img_idx]
                    base_image = img_data['base_image']
                    image_bytes = base_image["image"]

                    # Convert bytes to PIL Image
                    img = Image.open(io.BytesIO(image_bytes))
                    pil_images.append(img)

                # Try to arrange the parts in a grid
                try:
                    # Determine layout (simple approach)
                    if len(pil_images) <= 2:
                        # Horizontal layout for 1-2 images
                        total_width = sum(img.width for img in pil_images)
                        max_height = max(img.height for img in pil_images)

                        combined = Image.new('RGB', (total_width, max_height), (255, 255, 255))

                        x_offset = 0
                        for img in pil_images:
                            combined.paste(img, (x_offset, 0))
                            x_offset += img.width
                    else:
                        # Grid layout for 3+ images
                        cols = min(len(pil_images), 3)  # Max 3 columns
                        rows = (len(pil_images) + cols - 1) // cols

                        # Calculate sizes
                        max_width = max(img.width for img in pil_images)
                        max_height = max(img.height for img in pil_images)

                        combined = Image.new('RGB', (max_width * cols, max_height * rows), (255, 255, 255))

                        for i, img in enumerate(pil_images):
                            row = i // cols
                            col = i % cols
                            combined.paste(img, (col * max_width, row * max_height))

                    # Save combined figure
                    combined_path = f"{folder_path}/figure{page_index+1}_{group_idx+1}.png"
                    combined.save(combined_path)
                    print(f"[+] Combined figure saved as {combined_path}")

                except Exception as e:
                    print(f"[!] Error combining figure parts: {e}")
            else:
                # Just a single image
                img_data = image_data[group[0]]
                base_image = img_data['base_image']
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                # Save image
                image_name = f"{folder_path}/figure{page_index+1}_{group_idx+1}.{image_ext}"
                with open(image_name, "wb") as image_file:
                    image_file.write(image_bytes)
                print(f"[+] Figure saved as {image_name}")

            # Save caption if available
            if caption_text:
                caption_file = f"{folder_path}/figure{page_index+1}_{group_idx+1}_caption.txt"
                with open(caption_file, "w", encoding="utf-8") as f:
                    f.write(caption_text)
                caption_preview = caption_text[:50] + "..." if len(caption_text) > 50 else caption_text
                print(f"[+] Caption: {caption_preview}")
                total_captions += 1
            else:
                print(f"[!] No caption found for figure {page_index+1}_{group_idx+1}")

    print(f"Processed {file_path}: {total_figures} figures, {total_captions} captions found")

In [ ]:
def find_pdf_files(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

In [ ]:
output_base_dir = '/content/training_data_images'
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

# Choose where to save the final results in Google Drive
drive_output_path = '/content/drive/MyDrive/training_data_images'
if not os.path.exists(drive_output_path):
    os.makedirs(drive_output_path)

# Setup the sample directory for PDF files
sample_dir = '/content/sample_pdfs/sample'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Mount Google Drive
drive.mount('/content/drive')

# Upload a folder of PDFs using a zip file
print("Please upload a zip file containing your PDF files...")
uploaded = files.upload()  # Upload a zip file

# Extract the uploaded zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(sample_dir)
        print(f"Extracted {filename} to {sample_dir}")

# Process all PDFs in the sample directory - CORRECTED CODE HERE
pdf_files = find_pdf_files(sample_dir)
print(f"Found {len(pdf_files)} PDF files to process")

for filepath in tqdm(pdf_files):
    try:
        extract_images(filepath, output_base_dir)
    except Exception as e:
        print(f"An unexpected error occurred while processing {os.path.basename(filepath)}: {e}")

# Copy results back to Google Drive
shutil.copytree(output_base_dir, drive_output_path, dirs_exist_ok=True)
print(f"All extracted images and captions have been saved to Google Drive at: {drive_output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Please upload a zip file containing your PDF files...


Saving sample.zip to sample.zip
Extracted sample.zip to /content/sample_pdfs/sample
Found 100 PDF files to process


  0%|          | 0/100 [00:00<?, ?it/s]

[+] Combined figure saved as /content/training_data_images/1806.05575/figure6_1.png
[+] Caption: Figure 3. CIFAR-10: Real example images (left), sa...
[+] Figure saved as /content/training_data_images/1806.05575/figure7_1.png
[+] Caption: Figure 6. Small ImageNet inpainting examples. Left...
[+] Combined figure saved as /content/training_data_images/1806.05575/figure7_2.png
[+] Caption: tially outperforms the baseline in terms of ﬁnal p...
[+] Combined figure saved as /content/training_data_images/1806.05575/figure8_1.png
[+] Caption: Figure 7. Class-conditional samples from PixelIQN....
[+] Combined figure saved as /content/training_data_images/1806.05575/figure13_1.png
[+] Caption: Figure 9. CelebA 64x64: Real example images (left)...
[+] Figure saved as /content/training_data_images/1806.05575/figure14_1.png
[+] Caption: Figure 10. Samples from PixelIQN trained on small ...
[+] Combined figure saved as /content/training_data_images/1806.05575/figure15_1.png
[+] Caption: Figure 11. C

  1%|          | 1/100 [00:02<04:06,  2.49s/it]

...

 99%|█████████▉| 99/100 [03:23<00:11, 11.52s/it]

[+] Combined figure saved as /content/training_data_images/1805.09786/figure15_1.png
[+] Caption: Figure 9: A depiction of a hyperbolic recursive tr...
Processed /content/sample_pdfs/sample/sample/1805.09786.pdf: 4 figures, 4 captions found
[+] Combined figure saved as /content/training_data_images/1806.01822/figure5_1.png
[+] Caption: Figure 2: Tasks. We tested the RMC on a suite of s...
[+] Figure saved as /content/training_data_images/1806.01822/figure7_1.png
[+] Caption: Figure 3: Model analysis. Each row depicts the att...
[+] Figure saved as /content/training_data_images/1806.01822/figure15_1.png
[+] Caption: Figure 6: Samples of programmatic tasks. Note that...
[+] Figure saved as /content/training_data_images/1806.01822/figure15_2.png
[+] Caption: Figure 7: Memorization tasks. Each sub-task takes ...


100%|██████████| 100/100 [03:23<00:00,  2.04s/it]

[+] Figure saved as /content/training_data_images/1806.01822/figure16_1.png
[+] Caption: Figure 8: Programmatic results. From left to right...
[+] Figure saved as /content/training_data_images/1806.01822/figure16_2.png
[+] Caption: Figure 9: Example BoxWorld level. The left panel s...
[+] Combined figure saved as /content/training_data_images/1806.01822/figure17_1.png
[+] Caption: Figure 10: Mini Pacman Results.
Processed /content/sample_pdfs/sample/sample/1806.01822.pdf: 7 figures, 7 captions found


All extracted images and captions have been saved to Google Drive at: /content/drive/MyDrive/training_data_images
